In [1]:
import pandas as pd

In [2]:
labels = pd.read_pickle('labels_df')

IOError: [Errno 2] No such file or directory: 'labels_df'

In [ ]:
# convert labels to columns with probability or a 0.0
# make logistic regression model
# train test split accuracy
# apply model to current batch of docs and add results to pre-labeled probabilities